## Initialize RDKit environment

In [ ]:
! wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
! chmod +x Miniconda3-latest-Linux-x86_64.sh
! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
! conda install -q -y -c rdkit rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
!pip install git+https://github.com/keras-team/keras-tuner.git

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import warnings , os, re
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.utils import np_utils

from kerastuner.tuners import RandomSearch, BayesianOptimization
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters, Choice

from rdkit import Chem, rdBase
from rdkit.Chem import DataStructs, AllChem, RDConfig, rdMolDescriptors

#tf.get_logger().setLevel(logging.ERROR)
#tf.get_logger().setLevel('INFO')
#tf.autograph.set_verbosity(1)

np.set_printoptions(precision=3)

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Subroutines

In [ ]:
def readMolecule( sdfname ) :
    moles = Chem.rdmolfiles.SDMolSupplier( sdfname )
    return moles

def removeInactiveMolecules( moles, actname ) :
    moles = [ m for m in moles if None != m.GetProp( actname ) and float( m.GetProp( actname ) ) > 0 ]
    return moles

def getFingerprintFromMolecule( moles, nBits=2048 ) :
    fps = [ rdMolDescriptors.GetMorganFingerprintAsBitVect( m, 2, nBits=nBits ) for m in moles ]
    np_fps = []
    for fp in fps:
        arr = np.zeros( ( 1, ) )
        #DataStructs.ConvertToNumpyArray( fp, arr )
        DataStructs.cDataStructs.ConvertToNumpyArray( fp, arr )
        np_fps.append( arr )
    return np.array( np_fps )


def isActivityClass( m, activity ) :
    if re.search( activity, m.GetProp( 'Assay Description' ), re.IGNORECASE ) :
        return True
    return False

def findAgonist( moles ) :
    moles = [ m for m in moles if 'AGONIST' == m.GetProp( 'activity_class' ) ]
    return moles

def findAntagonist( moles ) :
    moles = [ m for m in moles if 'ANTAGONIST' == m.GetProp( 'activity_class' ) ]
    return moles



def getActivityOfMolecule( moles, actname ) :
    activity = [ m.GetProp( actname ) for m in moles ]
    activity = np.asarray( activity ).astype( 'float' )
    return activity


## DNN

In [ ]:
def prepareMoleculeFingerprint( path, nBits=2048 ) :
    sdfname = path + "200403-PPARg-ChEMBL-6101.sdf"
    actname = "pChEMBL_Value"
    moles = readMolecule( sdfname )
    num_load = len(moles)
    moles = removeInactiveMolecules( moles, actname )
    num_active = len(moles)
    # moles = findAgonist( moles )
    num_class = len(moles)
    print( "\n--->%d-moles loaded, %d-moles with pChEMBL_VALUE, %d-moles with activity_class\n" % (num_load, num_active, num_class ) )

    x = getFingerprintFromMolecule( moles, nBits=nBits )
    y = getActivityOfMolecule( moles, actname )

    return x,y

In [ ]:
def make_model( nfeatures=128 ):
    model = Sequential()
    model.add( Dense( nfeatures, input_dim=nfeatures, activation='relu' ) )
    model.add( Dense( nfeatures, activation='relu' ) )
    model.add( Dense(1) )
    model.compile( loss='mean_squared_error', optimizer='adam', metrics=['mae' ] )
    return model


In [ ]:
def plot_scatter( Y_train, Y_train_pred, Y_validation, Y_valid_pred ) :
    plt.figure(figsize=(6,6))
    plt.scatter(Y_train, Y_train_pred, color='black', s=2, label='training' )
    plt.scatter(Y_validation, Y_valid_pred, color='red', s=3, label='validation' )
    plt.xlabel( 'Experimental pIC50', labelpad=10 )
    plt.ylabel( 'Predicted pIC50', labelpad=10 )
    plt.xticks( np.arange( 2, 13 ) )
    plt.yticks( np.arange( 2, 13 ) )
    plt.savefig( 'qsar-scatter.png' )

def plot_history( history, filename='qsar-metric.png' ):
    _, loss_ax = plt.subplots( figsize=(6, 4 ) )
    loss_ax.set_ylim( [ 0, 4 ] )
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')

    acc_ax.plot(history.history['mae'], 'b', label='train MAE')
    acc_ax.plot(history.history['val_mae'], 'g', label='val MAE')
    acc_ax.set_ylabel('MAE')
    acc_ax.legend(loc='upper right')

    plt.savefig( filename )

In [ ]:
def do_regression( model, X_train, Y_train, X_validation, Y_validation, epochs=50, batch=32, graph=False ):
    hist = model.fit( X_train, Y_train, epochs=epochs, batch_size=batch, validation_split=0.2, verbose=0, callbacks=[  ] )
    loss_and_metric = model.evaluate( X_validation, Y_validation, verbose=0 )

    if( graph == True ) :
        #predictions = model.predict( X_validation )
        #print( hist.history.keys() )
        plot_history( hist )

        Y_train_pred = model.predict(X_train).flatten()
        Y_valid_pred = model.predict(X_validation).flatten()
        plot_scatter( Y_train, Y_train_pred, Y_validation, Y_valid_pred )

    return loss_and_metric, hist


def gridsearch( x, y, model ) :
    epochs = 30
    batchs = [ 16, 32, 64, 128, 256, 512 ]

    maxiter=10
    results = []
    for frac_test in np.arange( 0.1, 0.9, 0.1 ) :
        for batch in batchs :
            print( "===>frac_test=%.1f, batchsize=%2d" % ( frac_test, batch ) )
            for _ in np.arange(0, maxiter) :
                x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=frac_test )
                loss_and_metric, hist = do_regression( model, x_train, y_train, x_test, y_test, epochs, batch=batch )
                results.append( loss_and_metric )
                print( '%.1f, %3d : %.3f %.3f' % (frac_test, batch, loss_and_metric[0], loss_and_metric[1]), ' : ', "%.3f" % hist.history['loss'][-1], ', ', "%.3f" % hist.history['val_loss'][-1] )

    plt.figure( figsize=( 8, 6 ) )
    plt.ylim( 0, 1 )
    plt.plot( results, label=( [ 'loss', 'mae' ] ) )
    plt.savefig( 'qsar-metric-all.png' )
    return



def train( x, y, model, epochs = 50, batch = 64 ) :
    hist = model.fit( x, y, epochs=epochs, batch_size=batch, validation_split=0.2, verbose=1, callbacks=[] )
    plot_history( hist, 'qsar-train.png' )
    return model


def predict( model, sdfname ) :
    biocides = readMolecule( sdfname )
    x = getFingerprintFromMolecule( biocides, nBits=128 )
    y = model.predict(x).flatten()
    print(y)
    return y


In [ ]:
def parameter_gridsearch( x, y ) :
    nBits = x.shape[1]

    strategy = tf.distribute.MirroredStrategy()
    # print('\n===>Number of devices: {}\n'.format(strategy.num_replicas_in_sync))

    with strategy.scope() :
        model = make_model( nfeatures=nBits )

    maxiter=10
    epochs=50
    frac_tests = np.arange( 0.1, 0.9, 0.1 )
    batches = [ 16, 32, 64, 128, 256, 512 ]
    history = []
    for frac_test in frac_tests :
        for batch in batches :
            for _ in np.arange(0, maxiter) :
                hist = model.fit( x, y, epochs=epochs, batch_size=batch, validation_split=frac_test, verbose=0, callbacks=[  ] )
                history.append( [ hist.history['loss'][-1], hist.history['val_loss'][-1] ] )

    history = np.array(history)
    print( history )

    plt.figure( figsize=( 8, 6 ) )
    plt.ylim( 0, 1 )
    plt.plot( history[:,0], label='loss' )
    plt.plot( history[:,1], label='val_loss' )
    plt.savefig( 'qsar-metric-all.png' )
    return model


In [ ]:
def make_optimizer_model( nfeatures ) :
    # LR = Choice( 'learning_rate', values=[1e-2, 1e-3, 1e-4] )
    # MOMENTUM = Choice('momentum', values=[0.0, 0.2, 0.4, 0.6, 0.8, 0.9])
    optimizers = Choice('optimizer', values=['adam', 'sgd', 'rmsprop'] )

    model = Sequential()
    model.add( Dense( nfeatures, input_dim=nfeatures, activation='relu' ) )
    model.add( Dense( nfeatures, activation='relu' ) )
    model.add( Dense(1) )
    # model.compile( loss='mean_squared_error', optimizer=optimizers, metrics=['mae' ] )
    # model.compile( loss='mean_squared_error', optimizer=keras.optimizers.SGD( LR, momentum=MOMENTUM ), metrics=['mae' ] )
    # model.compile( loss='mean_squared_error', optimizer=keras.optimizers.Adam( LR ), metrics=['mae' ] )
    model.compile( loss='mean_squared_error', optimizer='adam', metrics=['mae' ] )
    return model



def hyperparam_optimization( x, y ) :
    nBits = x.shape[1]
    frac_test = 0.3
    x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=frac_test )

    TRAIN_EPOCHS = 20
    MAX_TRIALS=20
    EXECUTIONS_PER_TRIAL = 5

    b_tuner = BayesianOptimization(
        make_optimizer_model(nBits),
        objective = 'val_mean_squared_error',
        max_trials = MAX_TRIALS,
        executions_per_trial=EXECUTIONS_PER_TRIAL,
        directory='test_dir',
        project_name='tune_optimizer',
        seed=1
    )
    b_tuner.search_space_summary()

    b_tuner.search( x=x_train, y=y_train, epochs=TRAIN_EPOCHS, validation_data=(x_test, y_test))
    b_tuner.results_summary()
    best_model = b_tuner.get_best_models()[0]

    return best_model





In [ ]:
def train_chembl_predict_biocides( x, y, path ) :
    nBits = x.shape[1]

    strategy = tf.distribute.MirroredStrategy()
    # print('\n===>Number of devices: {}\n'.format(strategy.num_replicas_in_sync))

    with strategy.scope() :
        model = make_model( nfeatures=nBits )

    model = train( x, y, model, epochs = 10, batch = 64  )

    sdfname = "comp_182_ligand_prep_+_agonist.sd"
    predict( model, path + sdfname )
    return

In [ ]:
def main() :
    #config = tf.compat.v1.ConfigProto()
    #config.gpu_options.allow_growth = True
    #_ = tf.compat.v1.Session(config=config)

    # hyperparam_optimization( x, y, model )
    # gridsearch( x, y, model )
    # model = qsar( x, y, model )

    nBits = 128
    path = "./drive/My Drive/ColabNotebooks/"
    x, y = prepareMoleculeFingerprint( path, nBits=nBits )

    # hyperparam_optimization( x, y )
    parameter_gridsearch( x, y )
    # train_chembl_predict_biocides( x, y, path )

    return



if __name__ == '__main__' :
    main()
